In [ ]:
from encoder_decoder_correlations import EncoderDecoderComparison
import torch

from captum.attr import GradientShap, IntegratedGradients, Saliency

In [ ]:
# Define globals
DEVICE = 'gpu' if torch.cuda.is_available() else 'cpu'

## Gradshap

In [ ]:
gradient_shap_experiment = EncoderDecoderComparison(
    model_name="",
    attributer_factory=GradientShap,
    dataset='MNIST',
    device = DEVICE,
)

In [ ]:
res = gradient_shap_experiment.get_all_model_pearsons(True)

In [ ]:
print(res.mean(axis=1))

## Integrated Gradients

In [ ]:
ig_experiment = EncoderDecoderComparison(
    model_name="",
    attributer_factory=IntegratedGradients,
    dataset='MNIST',
    device = DEVICE,
)

In [ ]:
ig_pearsons = ig_experiment.get_all_model_pearsons(True)
ig_pearsons.mean(axis=1)

## Saliency

In [ ]:
sal_experiment = EncoderDecoderComparison(
    model_name="",
    attributer_factory=Saliency,
    dataset='MNIST',
    device = DEVICE,
)

In [ ]:
sal_pearsons = sal_experiment.get_all_model_pearsons(True)
sal_pearsons.mean(axis=1)